In [1]:
import fxcmpy
import pandas as pd
import datetime as dt
import json
import ta
import warnings

In [2]:
con = fxcmpy.fxcmpy(access_token="d808da394895605e414394e79c1a3c9ffafad862", log_level='error', server='demo', log_file='log.txt')

# return connection
if con.is_connected():
    print("Connect Successful!")

Connect Successful!


In [3]:
get_data = con.get_candles("EUR/USD", period='m5', number=50)[['bidopen','bidclose','bidhigh','bidlow']]
get_data.rename(columns = {'bidopen':'Open', 'bidhigh':'High', 'bidlow': 'Low', 'bidclose': 'Close'}, inplace = True)

In [30]:
res_ta = ta.trend.MACD(get_data['Close'], window_fast=12, window_slow=26 ,window_sign=9) # signal line is base and macd line will crossover
macd_line, signal_line = res_ta.macd()[-1:].values[0], res_ta.macd_signal()[-1:].values[0]

In [31]:
if macd_line > signal_line:
    signal, arrow_type = "BUY", "arrow-up"
elif macd_line < signal_line:
    signal, arrow_type = "SELL", "arrow-down"
else:
    signal, arrow_type = "NEUTRAL", "non-arrow"



('BUY', 'arrow-up')

In [35]:
res_ta = ta.momentum.RSIIndicator(get_data['Close'][-14:], window=14)

In [36]:
res_ta.rsi()

date
2022-05-03 10:40:00          NaN
2022-05-03 10:45:00          NaN
2022-05-03 10:50:00          NaN
2022-05-03 10:55:00          NaN
2022-05-03 11:00:00          NaN
2022-05-03 11:05:00          NaN
2022-05-03 11:10:00          NaN
2022-05-03 11:15:00          NaN
2022-05-03 11:20:00          NaN
2022-05-03 11:25:00          NaN
2022-05-03 11:30:00          NaN
2022-05-03 11:35:00          NaN
2022-05-03 11:40:00          NaN
2022-05-03 11:45:00    65.117649
Name: rsi, dtype: float64

In [14]:
indicators_signal = {
    "1": [
        {
            "timeframe": "10Min",
            "value": 1.05446,
            "signal": "SELL",
            "datetime": 1651592563000,
            "type": "trend"
        },
        {
            "timeframe": "1H",
            "value": 1.05213,
            "signal": "BUY",
            "datetime": 1651592563000,
            "type": "trend"
        }
    ],
    "2": [
        {
            "timeframe": "10Min",
            "arrowtype": "arrow-down",
            "value": 0.00009,
            "signal": "SELL",
            "datetime": 1651592563000,
            "type": "trend"
        },
        {
            "timeframe": "1H",
            "arrowtype": "arrow-up",
            "value": 0.00058,
            "signal": "BUY",
            "datetime": 1651592563000,
            "type": "trend"
        }
    ],
    "3": [
        {
            "timeframe": "10Min",
            "value": 32.5403,
            "signal": "NEUTRAL",
            "datetime": 1651592563000,
            "type": "momentum"
        },
        {
            "timeframe": "1H",
            "value": 54.35956,
            "signal": "NEUTRAL",
            "datetime": 1651592564000,
            "type": "momentum"
        }
    ]
}

In [49]:
return_summary = {}
return_summary['timeframe'] = {}
return_summary['all'] = {}
return_summary['summary'] = {}
return_summary['all']['SELL'], return_summary['all']['BUY'] = 0, 0

return_summary['trend'] = {}
return_summary['trend']['BUY'] = 0
return_summary['trend']['SELL'] = 0
return_summary['trend']['NEUTRAL'] = 0

return_summary['momentum'] = {}
return_summary['momentum']['OVERBOUGHT'] = 0
return_summary['momentum']['OVERSOLD'] = 0
return_summary['momentum']['NEUTRAL'] = 0
for indicator in indicators_signal.keys():

    for tf in indicators_signal[indicator]:

        if tf['timeframe'] not in return_summary['timeframe']:
            return_summary['timeframe'][tf['timeframe']] = {}
            # initial trend 
            return_summary['timeframe'][tf['timeframe']]['trend'] = {}
            return_summary['timeframe'][tf['timeframe']]['trend']['BUY'] = 0
            return_summary['timeframe'][tf['timeframe']]['trend']['SELL'] = 0
            return_summary['timeframe'][tf['timeframe']]['trend']['NEUTRAL'] = 0
            # initial momentum
            return_summary['timeframe'][tf['timeframe']]['momentum'] = {}
            return_summary['timeframe'][tf['timeframe']]['momentum']['OVERBOUGHT'] = 0
            return_summary['timeframe'][tf['timeframe']]['momentum']['OVERSOLD'] = 0
            return_summary['timeframe'][tf['timeframe']]['momentum']['NEUTRAL'] = 0

        if tf['type'] == 'trend':
            return_summary['timeframe'][tf['timeframe']]['trend'][tf['signal']] += 1
            return_summary['trend'][tf['signal']] += 1
        elif tf['type'] == 'momentum':
            return_summary['timeframe'][tf['timeframe']]['momentum'][tf['signal']] += 1
            return_summary['momentum'][tf['signal']] += 1

        if tf['signal'] in ['BUY', 'OVERSOLD']: return_summary['all']['BUY'] += 1
        elif tf['signal'] in ['SELL', 'OVERBOUGHT']: return_summary['all']['SELL'] += 1
    
if (return_summary['all']['BUY'] + return_summary['all']['SELL']) != 0:
    return_summary['summary']['BUY'] = return_summary['all']['BUY'] * 100 / (return_summary['all']['BUY'] + return_summary['all']['SELL'])
    return_summary['summary']['SELL'] = return_summary['all']['SELL'] * 100 / (return_summary['all']['BUY'] + return_summary['all']['SELL'])
else :
    return_summary['summary']['BUY'] = 0
    return_summary['summary']['SELL'] = 0

In [50]:
return_summary

{'timeframe': {'10Min': {'trend': {'BUY': 0, 'SELL': 2, 'NEUTRAL': 0},
   'momentum': {'OVERBOUGHT': 0, 'OVERSOLD': 0, 'NEUTRAL': 1}},
  '1H': {'trend': {'BUY': 2, 'SELL': 0, 'NEUTRAL': 0},
   'momentum': {'OVERBOUGHT': 0, 'OVERSOLD': 0, 'NEUTRAL': 1}}},
 'all': {'SELL': 2, 'BUY': 2},
 'summary': {'BUY': 50.0, 'SELL': 50.0},
 'trend': {'BUY': 2, 'SELL': 2, 'NEUTRAL': 0},
 'momentum': {'OVERBOUGHT': 0, 'OVERSOLD': 0, 'NEUTRAL': 2}}

In [29]:
high, low, close = get_data['High'], get_data['Low'], get_data['Close']


In [23]:

res_ta = ta.trend.PSARIndicator(high, low, close, step=float(0.02), max_step=float(0.2))
ta_value = res_ta.psar()[-1:].values[0]
ta_value

1.0540171746493372

In [19]:
warnings.filterwarnings("ignore")

In [37]:
res_ta = ta.trend.CCIIndicator(high, low, close, window=int(20), constant=float(0.015))
ta_value = res_ta.cci()[-1:].values[0]

In [39]:
# res_ta.cci()

In [40]:
res_ta = ta.momentum.StochasticOscillator(high, low, close, window=int(14), smooth_window=int(3))

In [44]:
indicator_value, indicators_signal = res_ta.stoch()[-1:], res_ta.stoch_signal()[-1:]
indicator_value, indicators_signal

(date
 2022-05-04 15:20:00    91.950464
 Name: stoch_k, dtype: float64,
 date
 2022-05-04 15:20:00    93.014905
 Name: stoch_k_signal, dtype: float64)

In [48]:
res_ta = ta.momentum.WilliamsRIndicator(high, low, close, lbp=14)
ta_value = res_ta.williams_r()[-1:].values[0]
ta_value

-8.049535603719619

In [3]:
print("W%R calculation error on > ")
import math

W%R calculation error on > 


In [14]:
start = dt.datetime(2022, 5, 1)
end = dt.datetime(2022, 5, 2)
data = con.get_candles("EUR/USD", period='m5', start=start, end=end)[['bidopen','bidclose','bidhigh','bidlow']]

In [17]:
data.index.view(int) ## - 162000, not more than 172800

array([1651439100000000000, 1651439400000000000, 1651439700000000000,
       1651440000000000000, 1651440300000000000, 1651440600000000000,
       1651440900000000000, 1651441200000000000, 1651442100000000000,
       1651442400000000000, 1651442700000000000, 1651443000000000000,
       1651443300000000000, 1651443600000000000, 1651443900000000000,
       1651444200000000000, 1651444500000000000, 1651444800000000000,
       1651445100000000000, 1651445400000000000, 1651446000000000000,
       1651446300000000000, 1651446600000000000, 1651446900000000000,
       1651447200000000000, 1651447500000000000, 1651447800000000000,
       1651448100000000000, 1651448400000000000, 1651448700000000000,
       1651449000000000000, 1651449300000000000, 1651449600000000000])

In [23]:
import datetime as dt

is_open = (math.floor(dt.datetime.utcnow().timestamp()) - 162000) % 604800 >= 172800
is_open

False